In [1]:
import os

# 환경 변수 설정 (시스템 내에 환경 변수가 없는 경우 코드에서 설정 가능)
os.environ['OPENAI_API_KEY'] = 'yourkey'

In [3]:
import os
import openai
import time

# 'client'라는 별칭으로 'openai' 모듈을 사용
client = openai
client.api_key = os.getenv('OPENAI_API_KEY')  # 환경 변수에서 API 키를 불러옵니다

# 전공 입력 검증
def validate_major(major):
    if not major or len(major.strip()) == 0:
        print("전공을 정확하게 입력해 주세요.")
        return False
    return True

# 답변 길이 및 내용 검사
def validate_answer_length(answer):
    if len(answer) < 50:
        print("답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.")
        return False
    elif len(answer) > 500:
        print("답변이 너무 깁니다. 간략하게 요약해 주세요.")
        return False
    return True

# 맞춤형 질문 생성 함수 (전공에 따라 난이도, 질문 유형 포함)
# general: 일반적인 질문 (자기소개, 장단점 등)
# technical: 기술적 질문 (프로그래밍, 알고리즘 등)
# behavioral: 행동 질문 (과거 경험, 대처 방법 등)
# experience: 경험 질문 (이전 프로젝트 경험 등)
def generate_custom_questions(major, difficulty='normal', question_types=['general', 'technical', 'behavioral', 'experience']):
    prompt = f"{major} 전공에 적합한 면접 질문을 {difficulty} 난이도로, "
    prompt += ', '.join(question_types) + " 유형의 질문을 각각 1개씩 생성해 주세요. (모두 한국어로 답변해주세요.)" # 한국어로 질문 할 수 있게

    # 난이도에 따른 질문을 다르게 생성
    if difficulty == 'easy':
        prompt += " 이 질문들은 기본적인 지식과 경험을 평가하는 질문입니다."
    elif difficulty == 'hard':
        prompt += " 이 질문들은 고급 기술적 지식과 문제 해결 능력을 평가하는 질문입니다."

    response = api_request_with_retry(prompt)
    if response:
        questions_text = response.choices[0].message.content.strip()
        return questions_text.split('\n')
    return []


# 사용 모델 - gpt-3.5-turbo 사용 이유는 gpt-4-turbo로 하면 질문이 이상함
# API 요청 최적화 (재시도 및 오류 처리)
def api_request_with_retry(prompt, retries=3, delay=2):
    for i in range(retries):
        try:
            # **파라미터 설정**
            response = client.chat.completions.create(
                model="gpt-3.5-turbo", # "gpt-4-turbo" 이거로 하면 질문이 이상함
                messages=[{'role': 'user', 'content': prompt}],
                temperature=0.7,  # 응답 생성의 무작위성을 제어 - 0에 가까울수록 응답이 더 보수적(예측 가능하고 비슷한 응답을 지속 생성할때 적합) / 1에 가까울수록 응답이 더 창의적이고 다양함(인터뷰나 창의적 답변 유도할 때 유리) - 면접 질문이여서 0.7로 적당한 창의성과 정확성의 균형을 위해 설정
                top_p=0.9, # 모델이 응답 생성 시 선택하는 단어의 확률 분포를 조정하는 파라미터로, 확률 상위의 단어만 선택해 응답을 구성 - 0에 가까울수록 흔히 반복적 / 1에 가까울수록 더 많은 선택지로부터 단어가 선택 - 그래서 1에 가깝게 설정
                frequency_penalty=0.5,  # 반복 단어를 줄이기 위한 패널티 - 0에 가까울수록 같은 단어가 더 자주 반복 / 1에 가까울수록 같은 단어가 덜 반복 - 균형 맞춤
                presence_penalty=0.5,   # 새로운 주제를 제시하도록 유도 - 0에 가까울수록 모델이 기존 맥락과 주제를 유지 / 1에 가까울수록 새로운 주제나 단어 도입 - 균형 맞춤
            )
            return response
        except openai.OpenAIError as e:
            if i < retries - 1:
                print(f"API 요청 실패, 재시도 중... ({i+1}/{retries})")
                time.sleep(delay)
            else:
                print(f"최대 재시도 횟수를 초과했습니다: {str(e)}")
                return None

# 피드백 생성 함수
def generate_feedback(responses):
    feedback_list = []

    for question, answer in responses:
        if len(answer.strip()) == 0:  # 답변이 없으면
            feedback_list.append({
                "text_feedback": "답변이 비어있습니다. 답변을 입력해 주세요.",
                "score": "0"
            })
        else:
            prompt = f"질문: {question}\n답변: {answer}\n\n이 답변에 대한 피드백을 제공하고 점수(1-10)를 매겨주세요."

            response = api_request_with_retry(prompt)
            if response:
                feedback_text = response.choices[0].message.content.strip()
                score = "점수 정보 없음"
                if "점수:" in feedback_text:
                    parts = feedback_text.split("점수:")
                    feedback_text = parts[0].strip()
                    score = parts[1].strip().split()[0] if parts[1].strip() else "점수 정보 없음"

                feedback_list.append({
                    "text_feedback": feedback_text,
                    "score": score
                })
            else:
                feedback_list.append({
                    "text_feedback": "피드백 생성에 실패했습니다.",
                    "score": "N/A"
                })

    return feedback_list

# 전반적인 면접 피드백 생성
def generate_overall_feedback(responses):
    # 빈 답변이 있으면 전체 피드백을 건너뛰기
    if any(len(answer.strip()) == 0 for _, answer in responses):
        return "빈 답변이 있어 전반적인 면접 피드백을 생성할 수 없습니다."

    overall_prompt = "전체 면접에 대한 피드백을 제공하고, 종합 점수를 매겨주세요."
    answers = "\n".join([f"질문: {q}\n답변: {a}" for q, a in responses])
    full_prompt = overall_prompt + "\n\n" + answers
    response = api_request_with_retry(full_prompt)
    if response:
        feedback_text = response.choices[0].message.content.strip()
        return feedback_text
    return "전반적인 면접 피드백을 생성할 수 없습니다."

# 답변 수집 및 피드백 생성 함수
def ask_question_and_get_feedback(major):
    if not validate_major(major):
        return

    difficulty = input("면접 질문의 난이도를 선택하세요 (easy/normal/hard): ")

    # 질문 유형 리스트

    # general: 일반적인 질문 (자기소개, 장단점 등)
    # technical: 기술적 질문 (프로그래밍, 알고리즘 등)
    # behavioral: 행동 질문 (과거 경험, 대처 방법 등)
    # experience: 경험 질문 (이전 프로젝트 경험 등)
    question_types = ['general', 'technical', 'behavioral', 'experience']

    # 여러 질문을 한 번에 생성
    questions = generate_custom_questions(major, difficulty, question_types)
    responses = []

    print(f"AI 면접 코칭 시스템에 오신 것을 환영합니다. 전공: {major}")

    for question in questions:
        print(f"질문: {question}")
        answer = input("답변: ")

        # 답변 길이 및 내용 검사
        if not validate_answer_length(answer):
            continue

        responses.append((question, answer))

    feedback = generate_feedback(responses)

    for i, (question, answer) in enumerate(responses):
        print(f"질문 {i + 1}: {question}")
        print(f"답변: {answer}")
        print(f"피드백: {feedback[i]['text_feedback']}")
        print(f"점수: {feedback[i]['score']}\n")

    overall_feedback = generate_overall_feedback(responses)
    print(f"전반적인 면접 피드백: {overall_feedback}")


In [4]:
# 프로그램 실행 예시: 전공을 입력받아 함수 호출
major = input("면접 준비 중인 전공을 입력하세요: ")
ask_question_and_get_feedback(major)

# 맞춤형 질문 생성 함수 (전공에 따라 난이도, 질문 유형 포함)
# general: 일반적인 질문 (자기소개, 장단점 등) - 인공지능 분야의 발전 가능성과 미래에 대한 기대감이 저에게 큰 동기부여가 되었습니다. 데이터와 학습 알고리즘을 통해 문제를 해결하는 과정은 매우 흥미로우며, 도전적이기도 합니다. 특히, 사람의 인지 능력을 모방하고 더 나아가 이를 넘어서는 기술을 탐구하고 개발하는 과정에서 지적 호기심과 성취감을 느끼게 되었습니다.
# technical: 기술적 질문 (프로그래밍, 알고리즘 등) - 현재 주요하게 관심을 가지고 있는 인공지능 기술 중 하나는 컴퓨터 비전입니다. 컴퓨터 비전은 기계가 이미지를 보고 해석하는 능력을 개발하는 분야로, 이미지 및 비디오 데이터를 분석하고 이해하여 특정 작업을 수행할 수 있도록 돕는 기술입니다. 사람의 시각적인 인지 과정을 모방하여, 사진이나 비디오에서 객체를 인식하거나, 장면을 분석하고, 상황을 파악하는 기술을 말합니다.
# behavioral: 행동 질문 (과거 경험, 대처 방법 등)
# experience: 경험 질문 (이전 프로젝트 경험 등)

면접 준비 중인 전공을 입력하세요: 인공지능
면접 질문의 난이도를 선택하세요 (easy/normal/hard): easy
AI 면접 코칭 시스템에 오신 것을 환영합니다. 전공: 인공지능
질문: 1. General: 인공지능에 대해 어떻게 정의하시겠습니까?
답변: 인공지능 분야의 발전 가능성과 미래에 대한 기대감이 저에게 큰 동기부여가 되었습니다. 데이터와 학습 알고리즘을 통해 문제를 해결하는 과정은 매우 흥미로우며, 도전적이기도 합니다. 특히, 사람의 인지 능력을 모방하고 더 나아가 이를 넘어서는 기술을 탐구하고 개발하는 과정에서 지적 호기심과 성취감을 느끼게 되었습니다.
질문: 2. Technical: 머신러닝과 딥러닝의 차이는 무엇인가요?
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
질문: 3. Behavioral: 이전 프로젝트에서 어떤 어려움을 겪었고, 그 문제를 해결하기 위해 어떤 노력을 기울였나요?
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
질문: 4. Experience: 지금까지 가장 만족스러웠던 인공지능 관련 프로젝트는 무엇이며, 그 경험에서 무엇을 배웠나요?
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
질문 1: 1. General: 인공지능에 대해 어떻게 정의하시겠습니까?
답변: 인공지능 분야의 발전 가능성과 미래에 대한 기대감이 저에게 큰 동기부여가 되었습니다. 데이터와 학습 알고리즘을 통해 문제를 해결하는 과정은 매우 흥미로우며, 도전적이기도 합니다. 특히, 사람의 인지 능력을 모방하고 더 나아가 이를 넘어서는 기술을 탐구하고 개발하는 과정에서 지적 호기심과 성취감을 느끼게 되었습니다.
피드백: 피드백: 답변은 인공지능에 대한 열정과 관심을 잘 보여주고 있습니다. 그러나, 주어진 질문에 대한 명확한 정의가 부족하다는 점이 있습니다. 좀 더 구체적으로 인공지능을 "인간 지능을 모방하거나 초월하는 컴퓨터 시스템 또는 소프트웨어"로 정의할 수 있습니다. 또한, 

In [5]:
# 프로그램 실행 예시: 전공을 입력받아 함수 호출
major = input("면접 준비 중인 전공을 입력하세요: ")
ask_question_and_get_feedback(major)

# 맞춤형 질문 생성 함수 (전공에 따라 난이도, 질문 유형 포함)
# general: 일반적인 질문 (자기소개, 장단점 등) - 인공지능 분야의 발전 가능성과 미래에 대한 기대감이 저에게 큰 동기부여가 되었습니다. 데이터와 학습 알고리즘을 통해 문제를 해결하는 과정은 매우 흥미로우며, 도전적이기도 합니다. 특히, 사람의 인지 능력을 모방하고 더 나아가 이를 넘어서는 기술을 탐구하고 개발하는 과정에서 지적 호기심과 성취감을 느끼게 되었습니다.
# technical: 기술적 질문 (프로그래밍, 알고리즘 등) - 현재 주요하게 관심을 가지고 있는 인공지능 기술 중 하나는 컴퓨터 비전입니다. 컴퓨터 비전은 기계가 이미지를 보고 해석하는 능력을 개발하는 분야로, 이미지 및 비디오 데이터를 분석하고 이해하여 특정 작업을 수행할 수 있도록 돕는 기술입니다. 사람의 시각적인 인지 과정을 모방하여, 사진이나 비디오에서 객체를 인식하거나, 장면을 분석하고, 상황을 파악하는 기술을 말합니다.
# behavioral: 행동 질문 (과거 경험, 대처 방법 등)
# experience: 경험 질문 (이전 프로젝트 경험 등)

면접 준비 중인 전공을 입력하세요: 인공지능
면접 질문의 난이도를 선택하세요 (easy/normal/hard): normal
AI 면접 코칭 시스템에 오신 것을 환영합니다. 전공: 인공지능
질문: 1. General: 인공지능에 대해 어떤 관심을 갖게 되었나요?
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
질문: 2. Technical: 다양한 머신러닝 알고리즘 중에서 어떤 것을 주로 사용하고 있으며, 그 이유는 무엇인가요?
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
질문: 3. Behavioral: 팀 프로젝트 중에 의견이 충돌했던 경험이 있다면, 그 상황을 설명해주시고 어떻게 해결했는지 알려주세요.
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
질문: 4. Experience: 이전 프로젝트에서 사용한 데이터셋이나 모델의 성능 향상을 위해 시도한 방법과 결과에 대해 이야기해주세요.
답변: ㅇㅇ
답변이 너무 짧습니다. 더 상세하게 답변을 작성해 주세요.
전반적인 면접 피드백: 이번 면접에 대한 피드백은 다음과 같습니다.

1. 의사소통능력: 면접 중 자신의 생각을 명확하게 표현하는 능력이 뛰어났으며, 질문에 대한 답변을 구체적으로 제시해 주셨습니다. 좋은 의사소통능력을 보여주었습니다.

2. 전문지식: 자신의 전문 분야에 대한 지식이 풍부하고, 관련 경험을 잘 설명해 주셨습니다. 면접 질문에 대한 전문적인 답변을 제공해 주셔서 긍정적인 인상을 줬습니다.

3. 태도 및 열정: 면접 동안 자신의 업무에 대한 열정과 책임감을 보여주었으며, 회사에 대한 관심과 적극적인 태도를 보여주었습니다. 

4. 자기소개 및 경력 소개: 자기소개와 이력서 소개가 명확하고 간결하게 이루어졌으며, 경력 사항을 구체적으로 설명해 주셨습니다.

종합적으로 본 면접에서는 매우 긍정적인 인상을 받았습니다. 의사소통능력, 전문지식, 태도 등 다양한 면에서 우수한 성과를 보였습니다. 따라서 종합 점수로 9/10

In [ ]:
# 구현하며 발생한 문제점
# 1. AI의 기본적인 긍정적 반응 경향: AI는 보통 학습된 대규모 텍스트 데이터에서 (사용자에게 친절하고 긍정적으로 반응하는 경향)
# 2. 평가 기준 부재: AI가 답변의 내용과 깊이를 제대로 평가하려면 (구체적 평가 기준이나 가이드라인이 필요)
# 3. 언어 모델의 환각적 피드백: AI가 피드백을 생성하는 과정에서 (실제 응답을 판단하는 대신, 일반적인 피드백 구조에 따라 '잘했다'는 식의 긍정적인 평가)

In [ ]:
# 개선활동
# 1. 비판적 피드백 학습 데이터 추가: 긍정적 피드백뿐만 아니라 비판적 피드백 예시가 포함된 데이터를 추가
# 2. 구체적인 평가 기준 마련: 면접 피드백 시스템의 경우, 응답의 길이, 논리성, 구체성, 질문 적합성 등의 평가 기준을 미리 정하고, 각 기준에 따라 피드백을 생성하도록 설정
# 3. 모범 답변 기준 데이터베이스 구축: 모범 답변 및 피드백 예시가 포함된 데이터베이스를 구축하여, 모델이 피드백을 생성할 때 이를 참조할 수 있도록 합니다.

In [ ]:
# 추가 api
# 1. Google Speech-to Text API - 음성을 텍스트로 변환하고 텍스트를 분석하여 피드백 제공(실제 면접에서 말하기와 비슷한 환경을 조성)
# 2. Microsoft Azure Text Analytics API - 감정 분석과 논리 평가 강화(사용자는 감정 및 논리적 피드백을 통해 자신의 면접 태도와 대화의 구조적 문제를 인식하고 개선)
# 3. Firebase Realtime Database - 연습 기록 저장 및 진행 상황 추적(이러한 기록은 사용자에게 반복 학습을 가능하게 하며, 과거의 오류를 다시 학습해 개선할 수 있는 기회를 제공)